Обычные модели GNN не могут быть использованы для работы с гетерографами, т.к. атрибуты разных типов узлов/связей не могут нормально обрабатываться одними и теми же функциями. 

Интуитивно очевидный вариант: реализовать независимые процедуры рассылки сообщений для каждого типа связей.

PyG предоставляет три способа создания моделей для работы с гетерографами:
1. Автоматическое преобразование обычной модели к модели, работающий с гетерографами (`nn.to_hetero()`, `nn.to_hetero_with_bases()`)
2. Описаний функций для разных типов связей при помощи `nn.conv.HeteroConv`;
3. Использование готовых или создание новых гетерогенных операторов.

## Автоматическое преобразование

PyG дает возможность автоматически конвертировать любую PyG GNN модель в гетерогенную модель.

In [1]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
import torch

E:\anaconda3\envs\gcn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        # PyG позволяет использовать отложенную инициализацию
        # размерностей; это удобно для гетерографов, где размерности
        # для разных типов могут меняться
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [3]:
dataset = OGB_MAG('./tmp', transform=T.ToUndirected(merge=True))
data = dataset[0]

In [4]:
@torch.no_grad()
def init_params():
    model(data.x_dict, data.edge_index_dict)

In [7]:
model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

init_params()

AttributeError: 'NoneType' object has no attribute 'dim'

In [6]:
data.x_dict

{'paper': tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.0616, -0.0277, -0.1338],
         [-0.1510, -0.1073, -0.2220,  ...,  0.3458, -0.0277, -0.2185],
         [-0.1148, -0.1760, -0.2606,  ...,  0.1731, -0.1564, -0.2780],
         ...,
         [ 0.0228, -0.0865,  0.0981,  ..., -0.0547, -0.2077, -0.2305],
         [-0.2891, -0.2029, -0.1525,  ...,  0.1042,  0.2041, -0.3528],
         [-0.0890, -0.0348, -0.2642,  ...,  0.2601, -0.0875, -0.5171]])}

In [1]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import ReLU
from tqdm import tqdm

import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import HGTLoader, NeighborLoader
from torch_geometric.nn import Linear, SAGEConv, Sequential, to_hetero



device = torch.device('cpu')


transform = T.ToUndirected(merge=True)
dataset = OGB_MAG('./tmp', transform=transform)

# Already send node features/labels to GPU for faster access during sampling:
data = dataset[0].to(device, 'x', 'y')

train_input_nodes = ('paper', data['paper'].train_mask)
val_input_nodes = ('paper', data['paper'].val_mask)
kwargs = {'batch_size': 1024, 'num_workers': 6, 'persistent_workers': True}


train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=True,
                                  input_nodes=train_input_nodes, **kwargs)
val_loader = NeighborLoader(data, num_neighbors=[10] * 2,
                            input_nodes=val_input_nodes, **kwargs)

model = Sequential('x, edge_index', [
    (SAGEConv((-1, -1), 64), 'x, edge_index -> x'),
    ReLU(inplace=True),
    (SAGEConv((-1, -1), 64), 'x, edge_index -> x'),
    ReLU(inplace=True),
    (Linear(-1, dataset.num_classes), 'x -> x'),
])
model = to_hetero(model, data.metadata(), aggr='sum').to(device)


# @torch.no_grad()
# def init_params():
#     # Initialize lazy parameters via forwarding a single batch to the model:
#     batch = next(iter(train_loader))
#     batch = batch.to(device, 'edge_index')
#     model(batch.x_dict, batch.edge_index_dict)


# init_params()  # Initialize parameters.

E:\anaconda3\envs\gcn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@torch.no_grad()
def init_params():
    # Initialize lazy parameters via forwarding a single batch to the model:
    batch = next(iter(train_loader))
    batch = batch.to(device, 'edge_index')
    model(batch.x_dict, batch.edge_index_dict)


init_params()  # Initialize parameters.

In [2]:
@torch.no_grad()
def init_params():
    # Initialize lazy parameters via forwarding a single batch to the model:
    model(data.x_dict, data.edge_index_dict)


init_params()  # Initialize parameters.

AttributeError: 'NoneType' object has no attribute 'dim'